# Final Group-5 Submittion "Medical Insurance Cost Prediction Using Machine Learning"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Load the sample database
df = pd.read_csv("/content/drive/MyDrive/ML_Project/insurance.csv")
df.describe()

# Cleaning data

### ENCODING CATEGORICAL FEATURES

Categorical Feautres :



1.   Sex
2.   Smoker
2.   Region




In [ ]:
# Create a label encoder object
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

# Create a label encoder object
le = LabelEncoder()

# Fit and transform the "category" column using the label encoder
df["sex"] = le.fit_transform(df["sex"])
category_map = {0: 1, 1: 0}
df["sex"] = df["sex"].map(category_map)

df["smoker"] = le.fit_transform(df["smoker"])
category_map = {0: 1, 1: 0}
df["smoker"] = df["smoker"].map(category_map)

# Create a sample dataframe
# 0=southeast
# 1=southwest
# 2=northeast
# 3=northwest

# # Create an ordinal encoder object
oe = OrdinalEncoder(categories=[['southeast', 'southwest', 'northeast','northwest']])

# # Fit and transform the "color" column using the ordinal encoder
df["region"] = oe.fit_transform(df[["region"]])

# # Set the value of the baseline category to 0
df.loc[df["region"] == "southeast", "region"] = 0

# Display the encoded dataframe
df['region'] = df['region'].astype(int)

# Display the encoded dataframe
print(df)

In [ ]:
# Check for missing values in the DataFrame
missing_values = df.isnull()
# Count the number of missing values in each column of the DataFrame
num_missing = missing_values.sum()

# Display the number of missing values in each column
print(num_missing)


> No missing values




In [ ]:
# Check for duplicates in the DataFrame
duplicates = df.duplicated()

# Count the number of duplicates in the DataFrame
num_duplicates = duplicates.sum()

# Display the number of duplicates in the DataFrame
print(num_duplicates)

In [ ]:
# Find the duplicate rows in the DataFrame
duplicate_rows = df[df.duplicated(keep=False)]

# Display the duplicate rows
print(duplicate_rows)



> Only 1 duplicate


In [ ]:
# Drop duplicates from the DataFrame
df = df.drop_duplicates()

# Display the first few rows of the DataFrame after dropping duplicates
print(df)

## Distributions

In [ ]:
#Display and plot The features to view Age distribution 
sns.set()
plt.figure(figsize=(6,6))
sns.histplot(df['age'])
plt.title('Age Distribution')
plt.show()



> The predominant age in this data set is between 18 and 20.




In [ ]:
#Display and plot The features to view Sex distribution 
plt.figure(figsize=(6,6))
sns.countplot(x='sex', data=df)
plt.title('Sex Distribution')
plt.show()

In [ ]:
#Display and plot The features to view BMI distribution 
plt.figure(figsize=(6,6))
sns.histplot(df['bmi'])
plt.title('BMI Distribution')
plt.show()

In [ ]:
#Display and plot The features to view Children distribution 
plt.figure(figsize=(6,6))
sns.countplot(x='children', data=df)
plt.title('Children')
plt.show()



> Most people in this dataset have no children and the maximum number of children is 5





In [ ]:
#Display and plot The features to view smoker distribution 
plt.figure(figsize=(6,6))
sns.countplot(x='smoker', data=df)
plt.title('smoker')
plt.show()

In [ ]:
#Display and plot The features to view region distribution 
plt.figure(figsize=(6,6))
sns.countplot(x='region', data=df)
plt.title('region')
plt.show()





> Most of the people in our dataset come from southeast region but it is very little differnce so it will be discarded



In [ ]:
#Display and plot The features to view expenses distribution 
plt.figure(figsize=(6,6))
sns.histplot(df['expenses'])
plt.title('Expenses Distribution')
plt.show()

## Box Plots

In [ ]:
# Create a box plot for expenses grouped by region
df.boxplot(column=["expenses"], by="region")

# Set the title of the plot
plt.title('Box plot of Medical Charges per Region')

# Clear the overall title of the plot
#plt.title('')
plt.suptitle('')

# Set the y-axis label
plt.ylabel('charges')
plt.xlabel('region')

# Display the plot
plt.show()

In [ ]:
# Create a box plot for expenses grouped by smoker
df.boxplot(column=["expenses"], by="smoker")

# Set the title of the plot
plt.title('Box plot of Medical Charges by Smoking status')

# Clear the overall title of the plot
#plt.title('')
plt.suptitle('')

# Set the y-axis label
plt.ylabel('charges')
plt.xlabel('smoker')

# Display the plot
plt.show()

In [ ]:
# Create a box plot for expenses grouped by region
df.boxplot(column=["expenses"], by="sex")

# Set the title of the plot
plt.title('Box plot of Medical Charges per Gender')

# Clear the overall title of the plot
#plt.title('')
plt.suptitle('')

# Set the y-axis label
plt.ylabel('charges')
plt.xlabel('sex')

# Display the plot
plt.show()

In [ ]:
# Create a box plot for expenses grouped by region
df.boxplot(column=["expenses"], by="children")

# Set the title of the plot
plt.title('Box plot of Medical Charges per children')

# Clear the overall title of the plot
#plt.title('')
plt.suptitle('')

# Set the y-axis label
plt.ylabel('charges')
plt.xlabel('children')

# Display the plot
plt.show()

## Feature extraction

In [ ]:
# Drop the 'region' column from the DataFrame
df.drop('region', axis=1, inplace=True)
print(df)

In [ ]:
#Define x and y; x represents all the column except region that was dropped earlier and the target feature,and y represents the target feature "Expenses" 
x = df.drop(columns='expenses', axis=1)
y = df['expenses']
print(df)

In [ ]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
normalize_scaler = MinMaxScaler()

# Fit and transform the data
normalized_data = normalize_scaler.fit_transform(df[df.columns])
df_norm = pd.DataFrame(normalized_data, columns=df.columns)
print(normalized_data)

In [ ]:
# Scale the data
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
scaled_data = scaler.fit_transform(df)
df_scaled = pd.DataFrame(scaled_data, columns=df.columns)
print(scaled_data)

## Box plots after Normalization and Scale

In [ ]:
# Create a boxplot of the scaled data frame
feature1 = df_scaled['age']
feature2 = df_scaled['bmi']
feature3 = df_scaled['children']
features_scaled = [feature1,feature2,feature3]

In [ ]:
# Create a boxplot of the normalize data frame
feature1 = df_norm['age']
feature2 = df_norm['bmi']
feature3 = df_norm['children']
features_normalized = [feature1,feature2,feature3]

In [ ]:
import seaborn as sns

# assuming 'data1' and 'data2' are your data
sns.set(style="ticks")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# plot the boxplot of the scaled data frame
sns.boxplot(data=features_scaled, ax=ax1)
ax1.set_xlabel('Features')
ax1.set_ylabel('Scaled value')
ax1.set_title('Boxplot of the scaled data frame')
#ax1.set_xticks(range(len(labels)))
ax1.set_xticklabels(['Age','Bmi','Children'])

# plot the boxplot of the scaled data frame
sns.boxplot(data=features_normalized, ax=ax2)
ax2.set_xlabel('Features')
ax2.set_ylabel('Normalized value')
ax2.set_title('Boxplot of the normalize data frame')
ax2.set_xticklabels(['Age','Bmi','Children'])

# adjust spacing between the plots
plt.subplots_adjust(wspace=0.3)

# show the plot
plt.show()


### sex and insurance expenses

In [ ]:
#Distribution of sex and their insurance expenses
sns.scatterplot(x=df['age'],y=df['expenses'])
sns.regplot(x=df['age'],y=df['expenses'])
plt.title('Scatterplot of ho')


> The regression line has apositive slope hence we could say that older people pay more insurance charges than younger people.



### BMI and insurance charges

In [ ]:
# BMI and insurance expenses
sns.scatterplot(x=df['bmi'], y=df['expenses'])



> We can see that people with high BMI pay more with regards to insurance charges. It may suggest that they are susceptible to diseases. We can going to add a regression line.





In [ ]:
sns.regplot(x=df['bmi'],y=df['expenses'])



> The positive slope in our regression line confirms our assumptions on a positive correlation between BMI and insurance charges.





### How smoking affects BMI and Insurance Charges.


In [ ]:
# How smoking affects BMI and Insurance Charges.
sns.scatterplot(x=df['bmi'],y=df['expenses'],hue=df['smoker'])
plt.title('Scatterplot of how smoking affects BMI and Insurance Charges')



> We can say that most people who smoke have a higher BMI than non-smokers and they also pay higher insurance charges than non-smokers.





In [ ]:
# creating train and test sets
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
# creating a Linear regression model
model = LinearRegression()
# fitting the model
model.fit(x_train,y_train)
# making predictions
Linear_predictions = model.predict(x_test)
#Calculating R2 square
r2_test = metrics.r2_score(y_test,Linear_predictions)
print(f"R squared value: {r2_test*100}")

In [ ]:
#Scatter plot between the test the predicted data 
sns.scatterplot(x=y_test,y= Linear_predictions, s=150)
plt.xlabel('y_test data')
plt.ylabel('Predictions')

In [ ]:
# create a decision tree regressor model
regressor = DecisionTreeRegressor() 
# fit the model
regressor.fit(x_train,y_train)
# making predictions
tree_predictions = regressor.predict(x_test)
#Calculating R2 square
r2_test = metrics.r2_score(y_test,tree_predictions)
print(f"R squared value: {r2_test*100}")

In [ ]:
#Scatter plot between the test the predicted data 
sns.scatterplot(x=y_test, y=tree_predictions, s=150)
plt.xlabel('y_test data')
plt.ylabel('Predictions')

In [ ]:
# create a gradient boosting regressor model
reg = GradientBoostingRegressor() 
# fit the model
reg.fit(x_train,y_train)
# making predictions
predictions = reg.predict(x_test)
#Calculating R2 square
r2_test = metrics.r2_score(y_test,predictions)
print(f"R squared value: {r2_test*100}")

In [ ]:
#Scatter plot between the test the predicted data 
sns.scatterplot(x=y_test,y= predictions, s=150)
plt.xlabel('y_test data')
plt.ylabel('Predictions')